In [1]:
# Import Dependencies 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


2024-06-01 12:29:08.827173: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Initialize the model
model = Sequential()

In [3]:
# Notes to consider upon this specific dataset
# There are two directories under cropped folder named test and train, 
# within each directory there are 120 categories of dogs
# within each directory there are 20,580 images of assorted dog breeds

In [4]:
# Adding in the first convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
# Add a max pooling layer
model.add(MaxPooling2D((2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
# Add another max pooling layer
model.add(MaxPooling2D((2, 2)))

# Add another convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
# Add another max pooling layer
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add a fully connected layer with 128 neurons
model.add(Dense(128, activation='relu'))

# Add the output layer with 120 neurons
model.add(Dense(120, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

/Users/armandocota/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        15,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 698,680 (2.67 MB)

 Trainable params: 698,680 (2.67 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# Define the directories for the model to train and test
train_dir = "../Resources/cropped/train"
test_dir = "../Resources/cropped/test"

# 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Flow training images in batches using train_datagen
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(64, 64),  # resize images to (64, 64)
        batch_size=batch_size,
        class_mode='categorical') 

# Flow validation images in batches using val_datagen
validation_generator = val_datagen.flow_from_directory(
        test_dir, 
        target_size=(64, 64), 
        batch_size=batch_size,
        class_mode='categorical') 

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples // batch_size)

# Evaluate the trained model
score = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])



Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.
Epoch 1/10


/Users/armandocota/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


375/375 ━━━━━━━━━━━━━━━━━━━━ 49s 127ms/step - accuracy: 0.0142 - loss: 4.7430 - val_accuracy: 0.0346 - val_loss: 4.4626
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 107us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 4.6104
Epoch 3/10


2024-06-01 12:30:05.502813: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/armandocota/anaconda3/envs/dev/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-06-01 12:30:05.522895: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


375/375 ━━━━━━━━━━━━━━━━━━━━ 48s 128ms/step - accuracy: 0.0409 - loss: 4.3442 - val_accuracy: 0.0592 - val_loss: 4.1954
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 46us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 4.1762
Epoch 5/10


...

375/375 ━━━━━━━━━━━━━━━━━━━━ 51s 136ms/step - accuracy: 0.1195 - loss: 3.7474 - val_accuracy: 0.1133 - val_loss: 3.8640
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 48us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 4.0251
Epoch 9/10


2024-06-01 12:32:33.490423: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:32:33.499482: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


375/375 ━━━━━━━━━━━━━━━━━━━━ 49s 129ms/step - accuracy: 0.1762 - loss: 3.4369 - val_accuracy: 0.1335 - val_loss: 3.8193
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 49us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 2.7213
  1/268 ━━━━━━━━━━━━━━━━━━━━ 18s 68ms/step - accuracy: 0.0312 - loss: 3.7255

2024-06-01 12:33:22.198186: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:33:22.207180: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.1259 - loss: 3.8404
Validation loss: 3.819382667541504
Validation accuracy: 0.13351212441921234


In [7]:
# Implementing the xception model
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [8]:
# loading in the pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False)

In [9]:
# Modify the model by adding custom top layers
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(120, activation='softmax'))


In [10]:
# Freeze the layers of the Xception model
base_model.trainable = False

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model using data generators
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model
score = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 98s 250ms/step - accuracy: 0.1026 - loss: 4.2225 - val_accuracy: 0.2006 - val_loss: 3.4945
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 4.3216
Epoch 3/10


2024-06-01 12:38:28.348400: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:38:28.396888: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


375/375 ━━━━━━━━━━━━━━━━━━━━ 100s 266ms/step - accuracy: 0.3015 - loss: 3.0628 - val_accuracy: 0.2271 - val_loss: 3.4673
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 183us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 3.8420
Epoch 5/10


2024-06-01 12:40:08.605581: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:40:08.649013: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


375/375 ━━━━━━━━━━━━━━━━━━━━ 104s 276ms/step - accuracy: 0.3713 - loss: 2.7382 - val_accuracy: 0.2359 - val_loss: 3.4082
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 174us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 2.6450
Epoch 7/10


...

375/375 ━━━━━━━━━━━━━━━━━━━━ 108s 287ms/step - accuracy: 0.4403 - loss: 2.4441 - val_accuracy: 0.2385 - val_loss: 3.4215
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 3.1959
Epoch 9/10


2024-06-01 12:43:40.679248: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:43:40.719425: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


375/375 ━━━━━━━━━━━━━━━━━━━━ 115s 305ms/step - accuracy: 0.4912 - loss: 2.2578 - val_accuracy: 0.2462 - val_loss: 3.3304
Epoch 10/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 3.0171


2024-06-01 12:45:35.610848: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-06-01 12:45:35.651771: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


268/268 ━━━━━━━━━━━━━━━━━━━━ 46s 170ms/step - accuracy: 0.2534 - loss: 3.2826
Validation loss: 3.329897165298462
Validation accuracy: 0.24638526141643524
